<a href="https://colab.research.google.com/github/daylightzjr/daylightzjr/blob/main/token_classfication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 命名实体识别

## step0 安装相关的库

In [40]:
!pip install transformers
!pip install datasets
!pip install torch
!pip install evaluate
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=6c8f20af5a2eb5f360576ed04d762d9baa7b0777be818b4ece6ed4611cd51b6b
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [41]:
##初始配置
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

##step1 导入相关的库

In [42]:
from transformers import AutoTokenizer, AutoModelForTokenClassification , TrainingArguments , Trainer,DataCollatorForTokenClassification
from datasets import load_dataset
import evaluate

##step2 加载数据集

In [43]:
dataset =  load_dataset("peoples_daily_ner", cache_dir="./data")

In [44]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 20865
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2319
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 4637
    })
})

In [45]:
dataset["train"][0]

{'id': '0',
 'tokens': ['海',
  '钓',
  '比',
  '赛',
  '地',
  '点',
  '在',
  '厦',
  '门',
  '与',
  '金',
  '门',
  '之',
  '间',
  '的',
  '海',
  '域',
  '。'],
 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0]}

In [46]:
dataset["train"].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}

In [47]:
label_list = dataset["train"].features["ner_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

##step3 数据集预处理

In [48]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


* is_split_into_words=True 是 Hugging Face 的 transformers 库中 tokenizer 函数的一个参数。它的作用是告知 tokenizer 输入的文本已经按单词（或子词）进行了分割，从而避免 tokenizer 再次进行分词操作。这个参数在处理已经分好词的列表时非常有用，尤其是当输入数据是由单独的词或子词组成的序列时。

In [49]:
sample = tokenizer(dataset["train"][0]["tokens"],is_split_into_words=True)
sample

{'input_ids': [101, 3862, 7157, 3683, 6612, 1765, 4157, 1762, 1336, 7305, 680, 7032, 7305, 722, 7313, 4638, 3862, 1818, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [50]:
tokenizer.decode(sample["input_ids"])

'[CLS] 海 钓 比 赛 地 点 在 厦 门 与 金 门 之 间 的 海 域 。 [SEP]'

1. token_type_ids（也称为 segment IDs）用于区分输入中的不同句子或段落。
2. attention_mask 是用于指示模型在计算时应该关注哪些位置。它主要用于处理句子中的填充（padding）部分。 attention_mask 的值为 1 的位置表示应该被模型关注（即实际的词），而值为 0 的位置表示不应该被关注（通常是填充的部分）。
3. input_ids 是将输入文本转换为词汇编号（token IDs）后的张量表示。每个 input_id 对应于模型词汇表（vocabulary）中的一个词（或子词）。input_ids 是对输入文本中每个词（或子词）在模型词汇表中索引的映射。例如，词“interesting”可能在词汇表中对应的索引为 1234，词“word”对应索引为 5678，那么 input_ids 就可能是 [101, 1234, 5678, 102]（假设 101 和 102 分别是句子的起始标志和结束标志）。

In [51]:
res = tokenizer("interesting word is")
res

{'input_ids': [101, 10673, 12865, 12921, 8181, 8681, 8310, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

1. Hugging Face 的 transformers 库中用于 BatchEncoding 对象的方法。它的主要作用是返回每个输入 token 对应的原始 word 的索引（在已分词的输入列表中）。这在处理命名实体识别（NER）或其他序列标注任务时尤其有用，因为它能够帮助我们将 token 与原始的 word 对齐。

In [52]:
res.word_ids()

[None, 0, 0, 0, 0, 1, 2, None]

In [53]:
#用word_ids实现标签映射
def process_function(examples):
  tokenized_examples = tokenizer(examples["tokens"],max_length=128,truncation=True,is_split_into_words=True)
  labels=[]
  for i,label in enumerate(examples["ner_tags"]):
    word_ids = tokenized_examples.word_ids(batch_index=i)
    label_ids = []
    for word_id in word_ids:
      if word_id is None:
        label_ids.append(-100)
      else:
        label_ids.append(label[word_id])
    labels.append(label_ids)
  tokenized_examples["labels"] = labels
  return tokenized_examples

* 遍历输入数据集中所有 example 的 NER 标签（ner_tags）。examples["ner_tags"] 是一个列表，其中每个元素是一个句子的标签列表（每个 word 的 NER 标签）

In [54]:
tokenized_datasets = dataset.map(process_function,batched=True)
tokenized_datasets

Map:   0%|          | 0/20865 [00:00<?, ? examples/s]

Map:   0%|          | 0/2319 [00:00<?, ? examples/s]

Map:   0%|          | 0/4637 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 20865
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2319
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 4637
    })
})

In [55]:
print(tokenized_datasets["train"][0])

{'id': '0', 'tokens': ['海', '钓', '比', '赛', '地', '点', '在', '厦', '门', '与', '金', '门', '之', '间', '的', '海', '域', '。'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0], 'input_ids': [101, 3862, 7157, 3683, 6612, 1765, 4157, 1762, 1336, 7305, 680, 7032, 7305, 722, 7313, 4638, 3862, 1818, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0, -100]}


##step4 创建模型

In [56]:
# 对于所有的非二分类任务，切记要指定num_labels，否则就会device错误
model = AutoModelForTokenClassification.from_pretrained("hfl/chinese-macbert-base",num_labels=len(label_list))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at hfl/chinese-macbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [57]:
#模型的相关的参数
model.config

BertConfig {
  "_name_or_path": "hfl/chinese-macbert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transfo

In [58]:
#表示该模型用于分类或序列标注任务时的标签（类别）数量
model.config.num_labels

7

## step5创建评估

In [59]:
sequval = evaluate.load("./seqeval_metric.py")
sequval

EvaluationModule(name: "seqeval", module_type: "metric", features: {'predictions': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence')}, usage: """
Produces labelling scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: List of List of predicted labels (Estimated targets as returned by a tagger)
    references: List of List of reference labels (Ground truth (correct) target values)
    suffix: True if the IOB prefix is after type, False otherwise. default: False
    scheme: Specify target tagging scheme. Should be one of ["IOB1", "IOB2", "IOE1", "IOE2", "IOBES", "BILOU"].
        default: None
    mode: Whether to count correct entity labels with incorrect I/B tags as true positives or not.
        If you want to only count exact matches, pass mode="strict". default: None.
    sample_weight: Array-like of sha

In [61]:
import numpy as np
def eval_metric(pred):
  predictions,labels = pred
  predictions = np.argmax(predictions , axis=-1)

  true_predictions=[
    [label_list[p] for p,l in zip(prediction,label) if l !=-100]
    for prediction,label in zip(predictions,labels)
  ]
  true_labels=[
    [label_list[l] for p,l in zip(prediction,label) if l !=-100]
    for prediction,label in zip(predictions,labels)
  ]

  result = sequval.compute(predictions=true_predictions,references=true_labels)

  return {
      "f1":result["overall_f1"]
  }


## step6 配置训练参数

In [62]:
args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    metric_for_best_model="f1",
    load_best_model_at_end=True,
    logging_steps=50,
    num_train_epochs=1
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


##step7 创建训练器

In [67]:
for param in model.parameters():
    param.data = param.data.contiguous()
trainer = Trainer(
    model=model,
    args = args,
    tokenizer = tokenizer,
    data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer),
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["validation"],
    compute_metrics = eval_metric
)

##step8 模型训练

In [68]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,0.026900,0.020953,0.945979


TrainOutput(global_step=1305, training_loss=0.007970030109087626, metrics={'train_runtime': 420.4867, 'train_samples_per_second': 49.621, 'train_steps_per_second': 3.104, 'total_flos': 1095359075955474.0, 'train_loss': 0.007970030109087626, 'epoch': 1.0})

In [69]:
trainer.evaluate(eval_dataset=tokenized_datasets["test"])

{'eval_loss': 0.027290990576148033,
 'eval_f1': 0.9362380446333687,
 'eval_runtime': 33.7358,
 'eval_samples_per_second': 137.45,
 'eval_steps_per_second': 8.596,
 'epoch': 1.0}

##step9 模型预测

In [70]:
from transformers import pipeline
model.config.id2label = {i:label for i,label in enumerate(label_list)}
model.config

BertConfig {
  "_name_or_path": "hfl/chinese-macbert-base",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  

1. aggregation_strategy 参数用于指定如何将模型对子词（subword）级别的预测结果聚合为词（word）级别的预测结果。它可以帮助我们将模型对一个词的不同子词的标签预测整合为一个标签，方便理解和使用
2. aggregation_strategy 的作用
aggregation_strategy 参数用于控制如何将这些子词的预测标签整合为一个词的标签。它的常用选项包括以下几种：

* "simple"（简单聚合策略）：
  这个策略会将一个词的所有子词的标签预测结果整合为该词第一个子词的预测结果。
* 例如，对于 ["un", "##believ", "##able"]，如果这三个子词的预测标签分别为 B-LOC, I-LOC, I-LOC，则整个词的标签会被聚合为 B-LOC。

In [71]:
# 对于NER任务，可以指定aggregation_strategy为simple，得到具体的实体的结果，而不是token的结果
pipe = pipeline("token-classification",model=model,tokenizer=tokenizer,aggregation_strategy="simple",device=0)


In [72]:
res  = pipe("中国是一个伟大的国家")
res

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity_group': 'LOC',
  'score': 0.9994892,
  'word': '中 国',
  'start': 0,
  'end': 2}]

In [73]:
# 根据start和end取实际的结果
ner_result = {}
x = "小明在北京上班"
for r in res:
    if r["entity_group"] not in ner_result:
        ner_result[r["entity_group"]] = []
    ner_result[r["entity_group"]].append(x[r["start"]: r["end"]])

ner_result

{'LOC': ['小明']}